In [1]:
import numpy as np
import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.stats import boxcox
from scipy import sparse
from sklearn.model_selection import StratifiedKFold
from itertools import product
import xgboost as xgb
from sklearn import preprocessing

from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.metrics import log_loss
import lightgbm as lgbm

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Kaggle 제출 용

#### 일반 데이터 셋

In [10]:
x_train = pd.read_csv('x_train.csv')
x_test = pd.read_csv('x_test.csv')
y_train = pd.read_csv('y_train.csv')
y_train.interest_level=y_train.interest_level.astype(int)
listing_id_df = pd.read_csv('test_listing_id.csv')
listing_id = listing_id_df['listing_id'].values

#### 프라이스 데이터 셋

In [4]:
x_train = pd.read_csv('x_train_price.csv')
x_test = pd.read_csv('x_test_price.csv')
y_train = pd.read_csv('y_train_price.csv')
y_train.interest_level=y_train.interest_level.astype(int)
listing_id_df = pd.read_csv('test_listing_id_price.csv')
listing_id = listing_id_df['listing_id'].values

#### v0.1 DataSet

In [26]:
x_train = pd.read_csv('x_train_v0.1.csv')
x_test = pd.read_csv('x_test_v0.1.csv')
y_train = pd.read_csv('y_train_v0.1.csv')
y_train.interest_level=y_train.interest_level.astype(int)
listing_id_df = pd.read_csv('test_listing_id_v0.1.csv')
listing_id = listing_id_df['listing_id'].values

#### 7번째 data set에 building ID만 채운 것

In [9]:
x_train = pd.read_csv('x_train_new_seventh.csv')
x_test = pd.read_csv('x_test_new_seventh.csv')
y_train = pd.read_csv('y_train_new_seventh.csv')
y_train.interest_level=y_train.interest_level.astype(int)
listing_id_df = pd.read_csv('test_listing_new_seventh.csv')
listing_id = listing_id_df['listing_id'].values

#### 7번째 data set

In [12]:
x_train = pd.read_csv('x_train_seventh.csv')
x_test = pd.read_csv('x_test_seventh.csv')
y_train = pd.read_csv('y_train_seventh.csv')
y_train.interest_level=y_train.interest_level.astype(int)
listing_id_df = pd.read_csv('test_listing_seventh.csv')
listing_id = listing_id_df['listing_id'].values

In [24]:
x_train = pd.read_csv('x_train_seventh_fillna.csv')
x_test = pd.read_csv('x_test_seventh_fillna.csv')
y_train = pd.read_csv('y_train_seventh_fillna.csv')
y_train.interest_level=y_train.interest_level.astype(int)
listing_id_df = pd.read_csv('test_listing_seventh_fillna.csv')
listing_id = listing_id_df['listing_id'].values

#### 8번째 데이터셋

In [4]:
x_train = pd.read_csv('x_train_eight.csv')
x_test = pd.read_csv('x_test_eight.csv')
y_train = pd.read_csv('y_train_eight.csv')
y_train.interest_level=y_train.interest_level.astype(int)
listing_id_df = pd.read_csv('test_listing_eight.csv')
listing_id = listing_id_df['listing_id'].values

#### 9번째 데이터셋

In [79]:
x_train = pd.read_csv('x_train_nineth.csv')
x_test = pd.read_csv('x_test_nineth.csv')
y_train = pd.read_csv('y_train_nineth.csv')
y_train.interest_level=y_train.interest_level.astype(int)
listing_id_df = pd.read_csv('test_listing_nineth.csv')
listing_id = listing_id_df['listing_id'].values

#### 10번째 데이터셋

In [202]:
x_train = pd.read_csv('x_train_tenth.csv')
x_test = pd.read_csv('x_test_tenth.csv')
y_train = pd.read_csv('y_train_tenth.csv')
y_train.interest_level=y_train.interest_level.astype(int)
listing_id_df = pd.read_csv('test_listing_tenth.csv')
listing_id = listing_id_df['listing_id'].values

#### 12번째 데이터셋

In [6]:
x_train = pd.read_csv('x_train_12th.csv')
x_test = pd.read_csv('x_test_12th.csv')
y_train = pd.read_csv('y_train_12th.csv')
y_train.interest_level=y_train.interest_level.astype(int)
listing_id_df = pd.read_csv('test_listing_12th.csv')
listing_id = listing_id_df['listing_id'].values

#### 13번째 데이터셋

In [6]:
x_train = pd.read_csv('x_train_16th.csv')
x_test = pd.read_csv('x_test_16th.csv')
y_train = pd.read_csv('y_train_16th.csv')
y_train.interest_level=y_train.interest_level.astype(int)
listing_id_df = pd.read_csv('test_listing_16th.csv')
listing_id = listing_id_df['listing_id'].values

In [7]:
x_train.shape

(49352, 295)

In [8]:
x_test.shape

(74659, 295)

# LightGBM for StackNet

In [43]:
ex_no = 120

train_file="stacked/train_stacknet_"+str(ex_no)+".csv"
test_file="stacked/test_stacknet_"+str(ex_no)+".csv"

 #Create Arrays for meta
train_stacker=[ [0.0 for s in range(3)]  for k in range (0,(x_train.shape[0])) ]
test_stacker=[[0.0 for s in range(3)]   for k in range (0,(x_test.shape[0]))]
        
cv_sum = 0
fpred = []
lgbm_rounds = []
NFOLDS = 5
cv_score_list = []
lgbm_params = {
    'boosting_type': 'gbdt', 'objective': 'multiclass', 'nthread': 4, 'silent': True,
    'num_leaves': 32, 'learning_rate': 0.03, 'max_depth': -1,
    'max_bin': 255, 'subsample_for_bin': 50000, 'num_class':3,'metric': 'multi_logloss',
    'subsample': 0.8, 'subsample_freq': 1, 'colsample_bytree': 0.7, 'reg_alpha': 1, 'reg_lambda': 1.2,
    'min_split_gain': 0.6, 'min_child_weight': 4, 'min_child_samples': 10, 'scale_pos_weight': 1}

kf = KFold(x_train.shape[0], n_folds=NFOLDS)
for i, (train_index, cross_index) in enumerate(kf):
    print('\n Fold %d' % (i+1))
    
    X_train, X_cross = x_train.iloc[train_index], x_train.iloc[cross_index]
    Y_train, Y_cross = y_train.iloc[train_index], y_train.iloc[cross_index]
    
    #X_train_temp,X_cross_temp = add_manager_skill(X_train,X_cross,Y_train)
    #W_train, W_cross = hcc_encode_wrapping(X_train,X_cross,Y_train,categorical)
    
    d_train = lgbm.Dataset(X_train, label=Y_train['interest_level'].values,silent=True)
    d_valid = lgbm.Dataset(X_cross, label=Y_cross['interest_level'].values,silent=True)
    #watchlist = [(d_train, 'train'), (d_valid, 'eval')]
    
    model = lgbm.train(lgbm_params, train_set=d_train, num_boost_round=100000, valid_sets=d_valid,
                        early_stopping_rounds=100, evals_result=None, verbose_eval=100, learning_rates=None, 
                        callbacks=None)
    lgbm_rounds.append(model.best_iteration)
    scores_val = model.predict(X_cross, num_iteration=model.best_iteration)
    cv_score = log_loss(Y_cross, scores_val)
    print('eval-MAE: %.6f' % cv_score)
    cv_score_list.append(cv_score)
    
    cv_sum = cv_sum + cv_score    
    
    reshaped_preds =scores_val.reshape( X_cross.shape[0], 3)
    
    no=0
    for real_index in cross_index:
        for d in range (0,3):
            train_stacker[real_index][d]=(reshaped_preds[no][d])
        no+=1
    
score = cv_sum / NFOLDS
print("Average Lolikelihood: ", score)
cv_index = 1
for cv_s in cv_score_list:
    print(cv_index,cv_s)
    cv_index+=1

print("test prediction")
bst_round = max(lgbm_rounds)
print("best round:", bst_round )

#x_train_temp,x_test_temp = add_manager_skill(x_train,x_test,y_train)
#W,W_test = hcc_encode_wrapping(x_train,x_test,y_train,categorical)
W = x_train.copy()
W_test = x_test.copy()
d_train_all = lgbm.Dataset(W, label=y_train['interest_level'].values,silent=True)
d_test = lgbm.Dataset(W_test)

model = lgbm.train(lgbm_params,d_train_all,bst_round)
predictions = model.predict(W_test)
preds=predictions.reshape( x_test.shape[0], 3)

for pr in range (0,len(preds)):  
    for d in range (0,3):            
        test_stacker[pr][d]=(preds[pr][d])

print("merging columns")
X=np.column_stack((W,train_stacker))
X_test=np.column_stack((W_test,test_stacker))   

# stack target to train
X=np.column_stack((y_train,X))
   
X_test=np.column_stack((listing_id,X_test))

print("exporting files")
np.savetxt(train_file, X, delimiter=",", fmt='%.5f')
np.savetxt(test_file, X_test, delimiter=",", fmt='%.5f')
                             
print("Write results...")
output_file = "output/submission_" + str( (score ))+"_ex" + str(ex_no) + ".csv"
print("Writing submission to %s" % output_file)
f = open(output_file, "w")   
f.write("listing_id,low,medium,high\n")# the header   
for g in range(0, len(test_stacker)):
    f.write("%s" % (listing_id[g]))
    for prediction in test_stacker[g]:
        f.write(",%f" % (prediction))    
    f.write("\n")
f.close()

print("fscore result")
fscore_df = pd.DataFrame(model.feature_importance())
fscore_df.rename(columns={0:'fscore'},inplace=True)
fscore_df['feature_name'] = model.feature_name()
fscore_df['fscore'] = fscore_df['fscore'].astype(int)
fscore_df.sort_values(by='fscore',ascending=False,inplace=True)
fscoe_output = 'fscore/ex_'+str(ex_no)+'_fscore.csv'
fscore_df.to_csv(fscoe_output)

#del x_train_temp,x_test_temp,W,W_test,X,X_test
print("Done.")


 Fold 1
Train until valid scores didn't improve in 100 rounds.
[100]	valid_0's multi_logloss: 0.597333
[200]	valid_0's multi_logloss: 0.557206
[300]	valid_0's multi_logloss: 0.543779
[400]	valid_0's multi_logloss: 0.537129
[500]	valid_0's multi_logloss: 0.533212
[600]	valid_0's multi_logloss: 0.531199
[700]	valid_0's multi_logloss: 0.529957
[800]	valid_0's multi_logloss: 0.529073
[900]	valid_0's multi_logloss: 0.528685
[1000]	valid_0's multi_logloss: 0.528216
[1100]	valid_0's multi_logloss: 0.528038
[1200]	valid_0's multi_logloss: 0.527944
Early stopping, best iteration is:
[1195]	valid_0's multi_logloss: 0.527907
eval-MAE: 0.529092

 Fold 2
Train until valid scores didn't improve in 100 rounds.
[100]	valid_0's multi_logloss: 0.565221
[200]	valid_0's multi_logloss: 0.521801
[300]	valid_0's multi_logloss: 0.507708
[400]	valid_0's multi_logloss: 0.50095
[500]	valid_0's multi_logloss: 0.496913
[600]	valid_0's multi_logloss: 0.494563
[700]	valid_0's multi_logloss: 0.49301
[800]	valid_0's 

# XGBOOST for StackNet

In [139]:
ex_no = 117

train_file="stacked/train_stacknet_"+str(ex_no)+".csv"
test_file="stacked/test_stacknet_"+str(ex_no)+".csv"

 #Create Arrays for meta
train_stacker=[ [0.0 for s in range(3)]  for k in range (0,(x_train.shape[0])) ]
test_stacker=[[0.0 for s in range(3)]   for k in range (0,(x_test.shape[0]))]
        
cv_sum = 0
fpred = []
xgb_rounds = []
NFOLDS = 5
cv_sum_list=[]
params = {
    'eta':0.03,
    'colsample_bytree':0.7,
    'subsample':0.8,
    'seed':0,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1,
    'min_child_weight':12,
    'max_depth':6,
    'gamma':0,
    'lambda':1,
    'alpha':1,
    'max_delta_step':0
}

kf = KFold(x_train.shape[0], n_folds=NFOLDS)
for i, (train_index, cross_index) in enumerate(kf):
    print('\n Fold %d' % (i+1))
    
    X_train, X_cross = x_train.iloc[train_index], x_train.iloc[cross_index]
    Y_train, Y_cross = y_train.iloc[train_index], y_train.iloc[cross_index]
    
    #X_train_temp,X_cross_temp = add_manager_skill(X_train,X_cross,Y_train)
    #W_train, W_cross = hcc_encode_wrapping(X_train_temp,X_cross_temp,Y_train,categorical)
    
    d_train = xgb.DMatrix(X_train, label=Y_train)
    d_valid = xgb.DMatrix(X_cross, label=Y_cross)
    watchlist = [(d_train, 'train'), (d_valid, 'eval')]
    
    bst = xgb.train(params,
                    d_train,
                    100000,
                    watchlist,
                    early_stopping_rounds=100,  
                    verbose_eval=25)
    
    xgb_rounds.append(bst.best_iteration)
    scores_val = bst.predict(d_valid, ntree_limit=bst.best_ntree_limit)
    cv_score = log_loss(Y_cross, scores_val)
    print('eval-MAE: %.6f' % cv_score)
    cv_sum_list.append(cv_score)
    cv_sum = cv_sum + cv_score    
    
    reshaped_preds =scores_val.reshape( X_cross.shape[0], 3)
    
    no=0
    for real_index in cross_index:
        for d in range (0,3):
            train_stacker[real_index][d]=(reshaped_preds[no][d])
        no+=1
    
score = cv_sum / NFOLDS
print("Average Lolikelihood: ", score)
cv_index = 1
for cv_s in cv_sum_list:
    print(cv_index,cv_s)
    
print("test prediction")
bst_round = max(xgb_rounds)
print("best round:", bst_round )

#x_train_temp,x_test_temp = add_manager_skill(x_train,x_test,y_train)
#W,W_test = hcc_encode_wrapping(x_train_temp,x_test_temp,y_train,categorical)

d_train_all = xgb.DMatrix(x_train, label=y_train)
d_test = xgb.DMatrix(x_test)

bst = xgb.train(params,d_train_all,bst_round)
predictions = bst.predict(d_test)
preds=predictions.reshape( x_test.shape[0], 3)

for pr in range (0,len(preds)):  
    for d in range (0,3):            
        test_stacker[pr][d]=(preds[pr][d])

print("merging columns")
X=np.column_stack((x_train,train_stacker))
X_test=np.column_stack((x_test,test_stacker))   

# stack target to train
X=np.column_stack((y_train,X))
   
X_test=np.column_stack((listing_id,X_test))

print("exporting files")
np.savetxt(train_file, X, delimiter=",", fmt='%.5f')
np.savetxt(test_file, X_test, delimiter=",", fmt='%.5f')
                             
print("Write results...")
output_file = "output/submission_" + str( (score ))+"_ex" + str(ex_no) + ".csv"
print("Writing submission to %s" % output_file)
f = open(output_file, "w")   
f.write("listing_id,low,medium,high\n")# the header   
for g in range(0, len(test_stacker)):
    f.write("%s" % (listing_id[g]))
    for prediction in test_stacker[g]:
        f.write(",%f" % (prediction))    
    f.write("\n")
f.close()

print("fscore result")
fscore_df = pd.DataFrame.from_dict(bst.get_fscore(),orient='index')
fscore_df.rename(columns={0:'fscore'},inplace=True)
fscore_df['fscore'] = fscore_df['fscore'].astype(int)
fscore_df.sort_values(by='fscore',ascending=False,inplace=True)
fscoe_output = 'fscore/ex_'+str(ex_no)+'_fscore.csv'
fscore_df.to_csv(fscoe_output)

del X,X_test
print("Done.")


 Fold 1
[0]	train-mlogloss:1.07742	eval-mlogloss:1.07812
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 100 rounds.
[25]	train-mlogloss:0.765209	eval-mlogloss:0.78453
[50]	train-mlogloss:0.640209	eval-mlogloss:0.673823
[75]	train-mlogloss:0.580436	eval-mlogloss:0.625023
[100]	train-mlogloss:0.547145	eval-mlogloss:0.599865
[125]	train-mlogloss:0.525777	eval-mlogloss:0.58467
[150]	train-mlogloss:0.510554	eval-mlogloss:0.574706
[175]	train-mlogloss:0.497881	eval-mlogloss:0.567403
[200]	train-mlogloss:0.486802	eval-mlogloss:0.561703
[225]	train-mlogloss:0.47711	eval-mlogloss:0.557117
[250]	train-mlogloss:0.468171	eval-mlogloss:0.553237
[275]	train-mlogloss:0.460033	eval-mlogloss:0.550065
[300]	train-mlogloss:0.452643	eval-mlogloss:0.547433
[325]	train-mlogloss:0.446549	eval-mlogloss:0.545101
[350]	train-mlogloss:0.439684	eval-mlogloss:0.542904
[375]	train-mlogloss:0.433674	eval-mlogloss:0.541178
[

# 각 CV마다 Stacking하여 제출

In [70]:
ex_no = 66

train_file="stacked/train_stacknet_"+str(ex_no)+".csv"
test_file="stacked/test_stacknet_"+str(ex_no)+".csv"

 #Create Arrays for meta
train_stacker=[ [0.0 for s in range(3)]  for k in range (0,(x_train.shape[0])) ]
test_stacker=[[0.0 for s in range(3)]   for k in range (0,(x_test.shape[0]))]
        
cv_sum = 0
fpred = []
xgb_rounds = []
NFOLDS = 5

params = {
    'eta':0.01,
    'colsample_bytree':0.7,
    'subsample':0.8,
    'seed':0,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1,
    'min_child_weight':10,
    'max_depth':6,
    'gamma':0,
    'lambda':1,
    'alpha':1,
    'max_delta_step':0
}

x_train_temp,x_test_temp = add_manager_skill(x_train,x_test,y_train)
W,W_test = hcc_encode_wrapping(x_train_temp,x_test_temp,y_train,categorical)

d_test = xgb.DMatrix(W_test)

kf = KFold(x_train.shape[0], n_folds=NFOLDS)
for i, (train_index, cross_index) in enumerate(kf):
    print('\n Fold %d' % (i+1))
    
    X_train, X_cross = x_train.iloc[train_index], x_train.iloc[cross_index]
    Y_train, Y_cross = y_train.iloc[train_index], y_train.iloc[cross_index]
    
    X_train_temp,X_cross_temp = add_manager_skill(X_train,X_cross,Y_train)
    W_train, W_cross = hcc_encode_wrapping(X_train_temp,X_cross_temp,Y_train,categorical)
    
    d_train = xgb.DMatrix(W_train, label=Y_train)
    d_valid = xgb.DMatrix(W_cross, label=Y_cross)
    watchlist = [(d_train, 'train'), (d_valid, 'eval')]
    
    bst = xgb.train(params,
                    d_train,
                    100000,
                    watchlist,
                    early_stopping_rounds=100,  
                    verbose_eval=25)
    
    xgb_rounds.append(bst.best_iteration)
    scores_val = bst.predict(d_valid, ntree_limit=bst.best_ntree_limit)
    cv_score = log_loss(Y_cross, scores_val)
    print('eval-MAE: %.6f' % cv_score)
    
    cv_sum = cv_sum + cv_score    
    
    reshaped_preds =scores_val.reshape( X_cross.shape[0], 3)
    
    no=0
    for real_index in cross_index:
        for d in range (0,3):
            train_stacker[real_index][d]=(reshaped_preds[no][d])
        no+=1
    
    print("test prediction")
    y_pred = bst.predict(d_test,ntree_limit=bst.best_ntree_limit)  
       
    preds=y_pred.reshape( x_test.shape[0], 3)
    for pr in range (0,len(preds)):  
        for d in range (0,3):            
            test_stacker[pr][d]= test_stacker[pr][d] + (preds[pr][d])
    
    if i > 0:
        fpred = pred + y_pred
    else:
        fpred = y_pred
    pred = fpred

score = cv_sum / NFOLDS
print("Average Lolikelihood: ", score)

bst_round = max(xgb_rounds)
print("best round:", bst_round )


print("merging columns")
# stack target to train
X=np.column_stack((W,train_stacker))
X=np.column_stack((y_train,X))

for pr in range (0,len(pred)):  
    for d in range (0,3):            
        test_stacker[pr][d]= test_stacker[pr][d] / NFOLDS
            
X_test=np.column_stack((W_test,test_stacker))   
X_test=np.column_stack((listing_id,X_test))

print("exporting files")
np.savetxt(train_file, X, delimiter=",", fmt='%.5f')
np.savetxt(test_file, X_test, delimiter=",", fmt='%.5f')
x                             
print("Write results...")
output_file = "output/submission_" + str( (score ))+"_ex" + str(ex_no) + ".csv"
print("Writing submission to %s" % output_file)

mpred = pred / NFOLDS
output_df = pd.DataFrame(mpred)
cols = ['low', 'medium', 'high']
output_df.columns = cols
output_df['listing_id'] = listing_id
output_df.to_csv(output_file,index=False)

print("fscore result")
fscore_df = pd.DataFrame.from_dict(bst.get_fscore(),orient='index')
fscore_df.rename(columns={0:'fscore'},inplace=True)
fscore_df['fscore'] = fscore_df['fscore'].astype(int)
fscore_df.sort_values(by='fscore',ascending=False,inplace=True)
fscoe_output = 'fscore/ex_'+str(ex_no)+'_fscore.csv'
fscore_df.to_csv(fscoe_output)

del x_train_temp,x_test_temp,W,W_test,X,X_test
print("Done.")

KeyboardInterrupt: 

## 사용하지 않는 Code

In [2]:
categorical4 = ['manager_id',
 'building_id','longi_lati']

categorical = ['manager_id',
 'building_id','display_address']

categorical3 = ['display_address',
 'manager_id',
 'building_id',
 'street_address','manager_id_building_id']

categorical2 = ['display_address',
 'manager_id',
 'building_id',
 'street_address',
 'display_address_manager_id',
 'display_address_building_id',
 'manager_id_building_id',
 'manager_id_street_address',
 'building_id_street_address']

def hcc_encode(train_df, test_df, variable, target, prior_prob, k, f=1, g=1, r_k=None, update_df=None):
    """
    See "A Preprocessing Scheme for High-Cardinality Categorical Attributes in
    Classification and Prediction Problems" by Daniele Micci-Barreca
    """
    hcc_name = "_".join(["hcc", variable, target])

    grouped = train_df.groupby(variable)[target].agg({"size": "size", "mean": "mean"})
    grouped["lambda"] = 1 / (g + np.exp((k - grouped["size"]) / f))
    grouped[hcc_name] = grouped["lambda"] * grouped["mean"] + (1 - grouped["lambda"]) * prior_prob

    df = test_df[[variable]].join(grouped, on=variable, how="left")[hcc_name].fillna(prior_prob)
    if r_k: df *= np.random.uniform(1 - r_k, 1 + r_k, len(test_df))     # Add uniform noise. Not mentioned in original paper

    if update_df is None: update_df = test_df
    if hcc_name not in update_df.columns: update_df[hcc_name] = np.nan
    update_df.update(df)
    return

def hcc_encode_wrapping(train_df,test_df,y_df,col_list):
    test_temp_df = test_df.copy()
    merged_df = pd.concat([train_df,y_df],axis=1)
    
    merged_df = merged_df.join(pd.get_dummies(merged_df["interest_level"], 
                                      prefix="pred").astype(int))
    
    prior_0, prior_1, prior_2 = merged_df[["pred_0", "pred_1", "pred_2"]].mean()
    
    skf = StratifiedKFold(5)
    attributes = product((col_list), zip(("pred_0","pred_1", "pred_2"), (prior_0, prior_1, prior_2)))
    
    for variable, (target, prior) in attributes:
        hcc_encode(merged_df, test_temp_df, variable, target, prior, k=5, r_k=None)
        for train, test in skf.split(np.zeros(len(merged_df)), merged_df['interest_level']):
            hcc_encode(merged_df.iloc[train], merged_df.iloc[test], variable, target, prior, k=5, r_k=0.01, update_df=merged_df)
            
    del merged_df['pred_0'], merged_df['pred_1'], merged_df['pred_2'], merged_df['interest_level']
    
    return merged_df,test_temp_df

In [3]:
def add_manager_skill(train,test,y_df):
    train_df = train.copy()
    test_df = test.copy()
    temp = pd.concat([train_df.manager_id,pd.get_dummies(y_df["interest_level"])], axis = 1).groupby('manager_id').mean()
    temp.columns = ['high_frac','low_frac', 'medium_frac']
    temp['count_frac'] = train_df.groupby('manager_id').count().iloc[:,1]
    temp['manager_skill'] = temp['high_frac']*2 + temp['medium_frac']

    # get ixes for unranked managers...
    unranked_managers_ixes = temp['count_frac']<20
    # ... and ranked ones
    ranked_managers_ixes = ~unranked_managers_ixes

    # compute mean values from ranked managers and assign them to unranked ones
    mean_values = temp.loc[ranked_managers_ixes, ['high_frac','low_frac', 'medium_frac','manager_skill']].mean()
    temp.loc[unranked_managers_ixes,['high_frac','low_frac', 'medium_frac','manager_skill']] = mean_values.values
    
    train_df_index = train_df.index
    train_df = train_df.merge(temp.reset_index(),how='left', left_on='manager_id', right_on='manager_id')
    train_df.index = train_df_index
    
    test_df_index = test_df.index
    test_df = test_df.merge(temp.reset_index(),how='left', left_on='manager_id', right_on='manager_id')
    test_df.loc[test_df['high_frac'].isnull(),['high_frac','low_frac', 'medium_frac','manager_skill']] = mean_values.values
    test_df.index = test_df_index
    
    not_use_col = ['high_frac', 'low_frac', 'medium_frac', 'count_frac']
    for col in not_use_col:
        del train_df[col]
        del test_df[col]
    
    return train_df,test_df